In [ ]:
# load required packages
!pip install pydub

import wave
import math
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
import wave
import pandas as pd
import numpy as np
import pydub
from scipy.io.wavfile import read
from pydub import AudioSegment
from pydub.utils import make_chunks
from os import walk
from os import listdir
from os.path import isfile, join
import random
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split,cross_validate,RepeatedStratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score,precision_recall_fscore_support
from sklearn.metrics import ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.preprocessing import MinMaxScaler,StandardScaler,normalize,LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold,GridSearchCV
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from numpy.linalg import eigh
from sklearn.svm import SVC
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from datetime import datetime
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from datetime import datetime

In [ ]:
# mount google drive to access files and folders
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# load the data

audiopath_df = pd.DataFrame(columns=['file','audio_class'])
cwd = os.getcwd()
folder = ['belly_pain', 'burping','discomfort','hungry','tired']
for t in folder:
  folder_path = cwd+'/gdrive/MyDrive/DS5500/audio/'+t
  audiofile_path = [folder_path+'/'+file for file in os.listdir(folder_path) if file.endswith('.wav')]
  file_df=pd.DataFrame({'file':audiofile_path,
                  'audio_class':t})
  audiopath_df = pd.concat([audiopath_df, file_df],ignore_index=True)

In [ ]:
# dataframe snippet

audiopath_df

,file,audio_class
0,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain
1,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain
2,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain
3,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain
4,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain
...,...,...
452,/content/gdrive/MyDrive/DS5500/audio/tired/B2A...,tired
453,/content/gdrive/MyDrive/DS5500/audio/tired/b4c...,tired
454,/content/gdrive/MyDrive/DS5500/audio/tired/B2A...,tired
455,/content/gdrive/MyDrive/DS5500/audio/tired/79F...,tired


In [ ]:
# extract length of file, age and gender of babies from the files and append to the dataframe

def calculate_audio_length(file_path):
    # load audio file using file path and calculate the length of audio in minutes
    y,sr = librosa.load(file_path)
    length = len(y)/sr
    return length

def extract_gender(file_path):
    # extract information on gender from file name
    g = file_path.split("-")[-3]
    if g == "m":
        return "Male"
    else: return "Female"

def extract_age(file_path):
    # extract information on age in month from file name
    a = int(file_path.split("-")[-2])
    return a

In [ ]:
# extract length, gender and age using above functions

audiopath_df["length"] = audiopath_df["file"].map(lambda x:calculate_audio_length(x))
audiopath_df["gender"] = audiopath_df["file"].map(lambda x:extract_gender(x))
audiopath_df["age_in_months"] = audiopath_df["file"].map(lambda x:extract_age(x))

In [ ]:
# dataframe snippet

audiopath_df.head(6)

,file,audio_class,length,gender,age_in_months
0,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain,6.84,Male,4
1,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain,7.00,Male,26
2,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain,7.00,Male,72
3,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain,7.00,Male,72
4,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain,6.82,Male,4
5,/content/gdrive/MyDrive/DS5500/audio/belly_pai...,belly_pain,7.00,Male,72


In [ ]:
# function for audio feature extraction

def processing_audio(audio_df = audiopath_df.head(), n_second = 1):
    i = 0
    gender = []
    age = []
    cry_reason = []
    mfccs = []
    spectral_centriod = []
    spectral_flatness = []
    spectral_contrast = []
    spectral_rolloff = []
    chroma_features = []
    audio_index = []
    for f in audio_df["file"]:
        y, s = librosa.load(f)
        while len(y)>= s*n_second:
            gender.append(f.split("-")[-3])
            age.append(f.split("-")[-2])
            cry_reason.append(audio_df["audio_class"][i])
            # Start from dividing the sound signal into the form of frames with a duration of 1-seconds
            f1 = y[:s*n_second]
            y = y[s*n_second:]
            # Extract Mel-frequency cepstral coefficients (MFCCs) to represent the short-term power spectrum of sound.
            mfcc = librosa.feature.mfcc(y=f1, sr=s, n_mfcc=19)
            mfccs.append(mfcc)
            # Extract Spectral Centriod
            cent = librosa.feature.spectral_centroid(y=f1, sr=s)
            spectral_centriod.append(cent)
            # Extract Spectral flatness
            flat = librosa.feature.spectral_flatness(y=f1)
            spectral_flatness.append(flat)
            # Extract Spectral contrast
            S = np.abs(librosa.stft(f1))
            sc = librosa.feature.spectral_contrast(S=S, sr=s)
            spectral_contrast.append(sc)
            # Extract Spectral Rolloff
            rolloff = librosa.feature.spectral_rolloff(y=f1, sr=s)
            spectral_rolloff.append(rolloff)
            # Extract Chroma Features
            cf = librosa.feature.chroma_stft(y=f1, sr=s)
            chroma_features.append(cf)
            audio_index.append(i)
        i += 1
    output_df = pd.DataFrame({"audio_index": audio_index,
                              "gender" : gender,
                              "age" : age,
                              "cry_reason" : cry_reason,
                              "mfcc" : mfccs,
                              "spectral_centroid": spectral_centriod,
                              "spectral_flatness": spectral_flatness,
                              "spectral_contrast": spectral_contrast,
                              "spectral_rolloff" : spectral_rolloff,
                              "chroma_features": chroma_features})
    return output_df

In [ ]:
def convert_matrix_vector_to_df(matrix_vector):
    # convert a vector of matrix to a vector of arrays by flattening all nested arrays
    flatten_df = matrix_vector.flatten()
    return flatten_df

In [ ]:
def convert_matrix_to_columns(data, column_name):
    # if the first cell of the column is a matrix then convert this column into multiple columns by flattening the matrix
    df = data[column_name].map(lambda x:convert_matrix_vector_to_df(x))
    temp_df = pd.DataFrame(df.tolist(),index = df.index)
    temp_df.columns = [column_name + f'{c}' for c in temp_df.columns]
    data = pd.concat([data,temp_df],axis =1)
    data = data.drop([column_name], axis=1)
    return data

In [ ]:
def feature_extraction(file_path_df):
    data_df = processing_audio(audio_df = file_path_df, n_second = 1)
    #2-D acoustic features are converted into 1-d vector before passing as input in the next process by flattening.
    flatten_data_df = data_df.copy()
    for column_name in data_df.columns:
        if type( data_df[column_name][0]) != str and not isinstance(data_df[column_name][0], (int, np.integer)) and (data_df[column_name][0].shape[0]*data_df[column_name][0].shape[1]>1):
            flatten_data_df = convert_matrix_to_columns(flatten_data_df, column_name)
    return data_df,flatten_data_df

In [ ]:
### Feature Extraction
data_df, flatten_data_df = feature_extraction(audiopath_df)

In [ ]:
# check the shape of the extracted features
print(data_df.columns)

print(data_df['mfcc'][0].shape)
print(data_df['spectral_centroid'][0].shape)
print(data_df['spectral_flatness'][0].shape)
print(data_df['spectral_contrast'][0].shape)
print(data_df['spectral_rolloff'][0].shape)
print(data_df['chroma_features'][0].shape)

Index(['audio_index', 'gender', 'age', 'cry_reason', 'mfcc',
       'spectral_centroid', 'spectral_flatness', 'spectral_contrast',
       'spectral_rolloff', 'chroma_features'],
      dtype='object')
(19, 44)
(1, 44)
(1, 44)
(7, 44)
(1, 44)
(12, 44)


In [ ]:
# check the datatypes of all columns

flatten_data_df.dtypes

audio_index             int64
gender                 object
age                    object
cry_reason             object
mfcc0                 float32
                       ...   
chroma_features523    float32
chroma_features524    float32
chroma_features525    float32
chroma_features526    float32
chroma_features527    float32
Length: 1808, dtype: object

In [ ]:
# convert age column to numerical, OHE for gender column

flatten_data_df['age'] = flatten_data_df['age'].astype('int')
data_df_dummies = pd.get_dummies(data=flatten_data_df, columns =['gender'], drop_first = True)
data_df_dummies

,audio_index,age,cry_reason,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,...,chroma_features519,chroma_features520,chroma_features521,chroma_features522,chroma_features523,chroma_features524,chroma_features525,chroma_features526,chroma_features527,gender_m
0,0,4,belly_pain,-466.562073,-397.598450,-373.749939,-363.782471,-372.105011,-387.254974,-410.950989,...,0.003087,0.006417,0.004836,0.001893,0.001346,0.003033,0.004589,0.021211,0.047314,1
1,0,4,belly_pain,-248.812759,-282.509277,-380.642944,-384.836395,-372.887909,-372.426361,-355.895569,...,0.005902,0.004062,0.003993,0.006552,0.004556,0.003969,0.004064,0.005336,0.011171,1
2,0,4,belly_pain,-233.144638,-253.146072,-322.028687,-315.527802,-317.190399,-336.630432,-348.778961,...,0.262661,0.177201,0.778797,0.823420,0.429188,0.106445,0.020027,0.004736,0.005901,1
3,0,4,belly_pain,-234.936981,-251.628754,-322.827118,-324.552521,-345.288666,-367.334229,-363.951111,...,0.104477,0.126058,0.103764,0.056916,0.048505,0.145680,0.345582,0.338705,0.256383,1
4,0,4,belly_pain,-438.140259,-416.819214,-394.987244,-391.403717,-392.398438,-359.272552,-339.711792,...,1.000000,1.000000,0.692701,1.000000,0.136667,0.550408,0.859743,0.380192,0.016092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944,456,26,tired,-340.772980,-357.969299,-429.610443,-440.136108,-421.517731,-367.844147,-346.111023,...,0.450458,1.000000,1.000000,0.752765,0.170682,0.466505,0.640135,0.384716,0.311756,1
2945,456,26,tired,-225.314926,-232.643784,-252.694687,-254.509537,-248.931259,-239.257416,-249.076508,...,0.541207,0.220463,0.201292,0.467611,0.543036,0.802548,0.499106,0.164619,0.185268,1
2946,456,26,tired,-369.406830,-363.462128,-374.100525,-378.500641,-387.281494,-383.564087,-370.970367,...,0.238718,0.251385,0.346880,0.136607,0.085671,0.151699,0.241273,0.273651,0.483385,1
2947,456,26,tired,-387.932037,-395.734039,-405.335571,-386.721558,-362.733521,-308.453522,-277.670685,...,1.000000,1.000000,0.636406,1.000000,1.000000,1.000000,1.000000,0.985155,0.746603,1


In [ ]:
# function for implementing Standard Scalar and PCA

def derive_pc(X_train,X_test):
    # input: X matrix
    # output: principal components that explain at least 90% of the variance

    # Standardize X_train and X_test for classification
    sc = StandardScaler()
    X_train_standardized = sc.fit_transform(X_train)
    X_test_standardized = sc.transform(X_test)

    # Calculate principal components
    pca = PCA(n_components= 0.90)
    pca.fit(X_train_standardized)
    X_train_pc = pca.transform(X_train_standardized)
    X_test_pc = pca.transform(X_test_standardized)

    print('n_components:', len(pca.components_))

    return X_train_pc, X_test_pc


In [ ]:
#Use SMOTE to oversample the minority class

def smote(X_train, y_train):
    oversample = SMOTE()
    over_X_train, over_y_train = oversample.fit_resample(X_train, y_train)
    return over_X_train, over_y_train

In [ ]:
# SVM with class_weight=balanced

def svm(X_train, X_test, y_train, y_test):
    # Derive principal components of X_train and X_test

    print('in the model')
    X_train_pc, X_test_pc = derive_pc(X_train, X_test)

    print('X_train_pc shape',X_train_pc.shape)
    print('X_test_pc shape',X_test_pc.shape)

    # label encoding
    le = LabelEncoder()
    y_train = to_categorical(le.fit_transform(y_train))
    y_test = to_categorical(le.transform(y_test))

    # Create the hyperparameter grid
    #kernels: calculate the similarity or distance between pairs of data points and transform data to higher dimensional space
    #gamma: determines the influence of a single training point. Mostly used for non-linear kernels like RBF
    #C: it is the regularization paramerter which determines penalty for misclassifying training examples.
    #degree: determines the degree of the polynomial used to compute the kernel function in case of polynomial kernel
    param_grid = [{'kernel': ['linear','rbf','poly','sigmoid'],
                    'gamma': [1e-2,1e-3],
                    'C': [1,10,100,1000],
                    'degree':[2,3]}]

    # Set the base model with class_weight as balanced
    svm_grid_model = SVC(class_weight='balanced')

    # Find the hyperparameter combination that returns the best result
    svm_grid_model_optimal = GridSearchCV(estimator = svm_grid_model,
                                        param_grid = param_grid,
                                        refit=True, # refit the model with best paramter on the dataset
                                        cv = 4,# 4-fold cross validation,
                                        verbose=2,# the computation time for each fold, parameter candidate, and the score are displayed
                                        n_jobs = -1) #using all processor

    # get the max prob class so input is 1D array
    y_train = np.argmax(y_train, axis = 1)

    # fit the model
    svm_grid_model_optimal.fit(X_train_pc, y_train)

    # Get the best SVM classifier from the grid search
    svm_best_model = svm_grid_model_optimal.best_estimator_

    print('best params',svm_grid_model_optimal.best_params_)

    # get test predictions
    predictions = svm_best_model.predict(X_test_pc)

    # we apply inverse of label encoder to get the corresponding lables of the numeric values
    y_pred_labels = le.inverse_transform(predictions)

    return y_pred_labels

In [ ]:
# SVM with Smote oversampling to train set

def svm_smote(X_train, X_test, y_train, y_test):

    print('in the model')
    # Derive principal components of X_train and X_test
    X_train_pc, X_test_pc = derive_pc(X_train, X_test)


    # apply smote to address class imbalance by oversampling minority class
    over_X_train, over_y_train = smote(X_train_pc, y_train)

    print('X_train_smote shape',over_X_train.shape)
    print('y_train_smote shape',over_y_train.shape)
    print('X_test shape',X_test_pc.shape)

    # label encoding
    le = LabelEncoder()
    y_train = to_categorical(le.fit_transform(over_y_train))
    y_test = to_categorical(le.transform(y_test))

    print('y_train le shape',y_train.shape)
    print('y_test le shape',y_test.shape)

    # Create the hyperparameter grid
    #kernels: calculate the similarity or distance between pairs of data points and transform data to higher dimensional space
    #gamma: determines the influence of a single training point. Mostly used for non-linear kernels like RBF
    #C: it is the regularization paramerter which determines penalty for misclassifying training examples.
    #degree: determines the degree of the polynomial used to compute the kernel function in case of polynomial kernel
    param_grid = [{'kernel': ['rbf','poly','sigmoid'],
                    'gamma': [1e-2,1e-3],
                    'C': [1,10,100],
                    'degree':[2,3]}]

    # Set the base model with class_weight as balanced
    svm_grid_model = SVC()

    # Find the hyperparameter combination that returns the best result
    svm_grid_model_optimal = GridSearchCV(estimator = svm_grid_model,
                                        param_grid = param_grid,
                                        refit=True, # refit the model with best paramter on the dataset
                                        cv = 2,# 4-fold cross validation,
                                        verbose=2,# the computation time for each fold, parameter candidate, and the score are displayed
                                        n_jobs = -1) #using all processor


    #svm_model = SVC()


    # get the max prob class so input is 1D array
    y_train = np.argmax(y_train, axis = 1)

    # fit the model
    svm_grid_model_optimal.fit(over_X_train, y_train)

    # Get the best SVM classifier from the grid search
    svm_best_model = svm_grid_model_optimal.best_estimator_

    print('best params',svm_grid_model_optimal.best_params_)

    # get test predictions
    predictions = svm_best_model.predict(X_test_pc)

    # we apply inverse of label encoder to get the corresponding lables of the numeric values
    y_pred_labels = le.inverse_transform(predictions)

    return y_pred_labels

In [ ]:
# Define function for reporting performance metrics

def report_performance(test_df,y_pred):
    # input: test data that contains both real label of cry reason and all features
    # output: a dictionary with metric names as keys.

    # calculate the number of audios by cry reason
    test_count = test_df.groupby(['cry_reason']).size()
    reason_dict = test_count.to_dict()

    # build a function that only keeps the most frequent cry reason from an array
    def parse_pred(entry):
        if isinstance(entry, str):
            return entry
        return max(entry, key=lambda x: reason_dict[x])
    pred_dict = {'audio_index': test_df['audio_index'], 'y_test': test_df['cry_reason'], 'y_pred': y_pred}
    temp_df = pd.DataFrame(pred_dict)

    # predict the entire audio's cry reason based on the majority group of predicted labels of its cuts
    temp_result = temp_df.groupby(['audio_index'])[['y_test','y_pred']].agg(pd.Series.mode)
    temp_result['y_pred'] = temp_result['y_pred'].map(parse_pred)

    # calculate precision, recall, and f1 based on the cry reason of orginal audio and predictions
    precision_1, recall_1,f1_1, null_value_1 = precision_recall_fscore_support(temp_result['y_test'],
                                                                               temp_result['y_pred'],
                                                                               average='macro',
                                                                               zero_division=0)

    precision_2, recall_2,f1_2, null_value_2 = precision_recall_fscore_support(temp_result['y_test'],
                                                                               temp_result['y_pred'],
                                                                               average='weighted',
                                                                               zero_division=0)
    # calucate accuracy rate

    accuracy = accuracy_score(temp_result['y_test'],temp_result['y_pred'])
    return {
        'precision_macro': precision_1,
        'recall_macro': recall_1,
        'f1_macro': f1_1,
        'precision_weighted': precision_2,
        'recall_weighted': recall_2,
        'f1_weighted': f1_2,
        'accuracy': accuracy
    }

In [ ]:
# Prep your inputs to the model, train test split
y = data_df_dummies['cry_reason']
X = data_df_dummies.copy()

print(X.shape)
print(y.shape)

# train-test-val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#X_test_new
X_test_new = X_test.drop(['cry_reason'],axis = 1)
X_train_new = X_train.drop(['cry_reason'],axis = 1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_test_new.shape)
print(X_train_new.shape)

print(y_train.shape)
print(y_test.shape)


(2949, 1808)
(2949,)
(2359, 1808)
(590, 1808)
(2359,)
(590,)
(590, 1807)
(2359, 1807)
(2359,)
(590,)


In [ ]:
# execute the models
models = ['svm(X_train_new, X_test_new, y_train, y_test)',
          'svm_smote(X_train_new, X_test_new, y_train, y_test)']


for model in models:
  print(model)
  y_pred = eval(model)
  metrics_dict = report_performance(X_test,y_pred)
  if model == models[0]:
    metrics_df = pd.Series(metrics_dict).to_frame().T
    metrics_df['model_name'] = model.split('(')[0]
  else:
    temp_df = pd.Series(metrics_dict).to_frame().T
    temp_df['model_name'] = model.split('(')[0]
    metrics_df = pd.concat([metrics_df, temp_df],axis = 0)

metrics_df


svm(X_train_new, X_test_new, y_train, y_test)
in the model
n_components: 281
X_train_pc shape (2359, 281)
X_test_pc shape (590, 281)
Fitting 4 folds for each of 64 candidates, totalling 256 fits
best params {'C': 1, 'degree': 2, 'gamma': 0.001, 'kernel': 'rbf'}
svm_smote(X_train_new, X_test_new, y_train, y_test)
in the model
n_components: 281
X_train_smote shape (9855, 281)
y_train_smote shape (9855,)
X_test shape (590, 281)
y_train le shape (9855, 5)
y_test le shape (590, 5)
Fitting 2 folds for each of 36 candidates, totalling 72 fits
best params {'C': 10, 'degree': 2, 'gamma': 0.001, 'kernel': 'rbf'}


,precision_macro,recall_macro,f1_macro,precision_weighted,recall_weighted,f1_weighted,accuracy,model_name
0,0.669322,0.264101,0.296454,0.832820,0.846377,0.789919,0.846377,svm
0,0.367442,0.208333,0.198278,0.768453,0.837681,0.766380,0.837681,svm_smote
